In [26]:
!pip3 install -q openai
!pip3 install -q Owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 27.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for Owlready2: filename=Owlready2-0.40-cp39-cp39-macosx_10_9_x86_64.whl size=23473484 sha256=0e1370bd648f11834434bbe8c6a2e649a7f3d1a47228ecec5f5c2b23b7d1c252
  Stored in directory: /Users/paulbruffett/Library/Caches/pip/wheels/d7/57/3d/e5c8b9956e87df35a535741844755214b6c6844d6a8a50dc76
Successfully built Owlready2


In [2]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai
from owlready2 import *

onto = get_ontology("http://test.org/onto.owl")

#add your API key or set it as an environment variable
openai.api_key = apikey

In [4]:
with onto:
    class Coffee(Thing): pass
            
    class Roast(Thing): pass
    
    class Dark_Roast(Roast): pass
    class Blonde_Roast(Roast): pass
    class Medium_Roast(Roast): pass
    
    class Region(Thing): pass
    
    class Latin_America(Region): pass
    class Asia_Pacific(Region): pass
    class Multi(Region): pass
    
    class Processing(Thing): pass
    
    class Washed(Processing): pass
    class Semi_Washed(Processing): pass
    

    class has_roast(Coffee >> Roast, FunctionalProperty):
        pass
    
    
    class from_region(Coffee >> Region, FunctionalProperty):
        pass
    
    #inverse relationships indicate the relationship is bi-directional
    class grown_in(Region >> Coffee):
        inverse = from_region
        
    class has_body(Coffee >> str, FunctionalProperty):
        pass
    
    class has_processing(Coffee >> Processing, ObjectProperty):
        pass
    
            
    class Western_Blend (Coffee):
        equivalent_to = [Coffee & has_roast.some(Roast) & has_roast.only(Blonde_Roast) &
                         from_region.some(Region) & 
                        from_region.only(Latin_America) & has_body.value("Light") 
                         & has_processing.some(Processing) & has_processing.only(Washed)]

    class Eastern_Blend(Coffee):
        equivalent_to = [Coffee & has_roast.some(Roast) & has_roast.only(Medium_Roast) &
                         from_region.some(Region) &
                        from_region.only(Latin_America) & has_body.value("Medium") &
                         has_processing.some(Processing) & has_processing.only(Washed)]

    class Southern_Blend(Coffee):
        equivalent_to = [Coffee & has_roast.some(Roast) & has_roast.only(Dark_Roast) &
                         from_region.some(Region) &
                        from_region.only(Asia_Pacific) & has_body.value("Full") &
                        has_processing.some(Processing) & has_processing.only(Washed)]
        
    #we can have two washes for this coffee, washed or semi-washed
    class Northern_Blend(Coffee):
        equivalent_to = [Coffee & has_roast.some(Roast) & has_roast.only(Medium_Roast) &
                        from_region.some(Region) & from_region.only(Multi) &
                        has_body.value("Full") & has_processing.some(Processing) &
                        has_processing.only(Washed | Semi_Washed)]


AllDisjoint([Dark_Roast, Blonde_Roast, Medium_Roast])
AllDifferent([Latin_America, Asia_Pacific, Multi])

#previously we related the unknown coffee to classes, now we're instantiating objects to relate to
latam = Latin_America("Latin America")
apac = Asia_Pacific("Asia Pacific")
multi = Multi("Multiple Regions")
blonde = Blonde_Roast("Blonde")
dark = Dark_Roast("Dark")
medium = Medium_Roast("Medium")

coffee1 = Coffee(name="Nomad", has_roast = blonde, from_region=latam, has_body = "Light",has_processing=[Washed()])
coffee2 = Coffee(name="Lucy", has_roast = dark, from_region=apac, has_body = "Full", has_processing=[Washed()])
coffee3 = Coffee(name="Monarch", has_roast = medium, from_region=multi, has_body = "Full", has_processing=[Washed(),
                                                                                                  Semi_Washed()])


close_world(Coffee)

In [5]:
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Users/paulbruffett/opt/anaconda3/lib/python3.9/site-packages/owlready2/hermit:/Users/paulbruffett/opt/anaconda3/lib/python3.9/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/4q/0r95mcrx4n5_z5935v7q04_c0000gn/T/tmpxzzku180
* Owlready2 * HermiT took 0.42424893379211426 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [6]:
coffee3.name

'Monarch'

In [31]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": "What's a light bodied washed coffee."}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-6tmjrQQ492DuoIu9XrF5SRbfjUSS2 at 0x7f9558157270> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\nAs an AI language model, I don't have personal preferences, but as per the coffee norms, A light-bodied washed coffee is a coffee that is processed by removing the fruit from the coffee cherry before drying. Once the fruit is removed, the beans are left to soak and ferment, which removes the sticky, sugary residue from the beans. This process results in a coffee that is often described as having a light or delicate body with bright, acidic, and citrusy notes. Examples of light-bodied washed coffees include Ethiopian Yirgacheffe and Kenyan AA.",
        "role": "assistant"
      }
    }
  ],
  "created": 1678753411,
  "id": "chatcmpl-6tmjrQQ492DuoIu9XrF5SRbfjUSS2",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 117,
    "pro

In [33]:
#{"prompt":"Item is a handbag. Colour is army green. Price is midrange. Size is small.->", "completion":" 
# This stylish small green handbag will add a unique touch to your look, without costing you a fortune."}

In [77]:
prompts = []
for i in [coffee1,coffee2,coffee3]:
    prompt_text="%s is the name of a coffee.  Body is %s.  From Region %s.  Roast is %s. ->" % (i.name, i.has_body, i.from_region.name,i.has_roast.name)
    completion_text="%s is a %s bodied coffee that is from %s that has been prepared with a %s roast. END" % (i.name, i.has_body, i.from_region.name,i.has_roast.name)
    prompts.append({"prompt":prompt_text, "completion":completion_text})
    prompt_text="What is the name of a good coffee with %s body? ->" % (i.has_body)
    completion_text="%s is a coffee with %s body. END" % (i.name, i.has_body)
    prompts.append({"prompt":prompt_text, "completion":completion_text})
    prompt_text="What is the name of a coffee from %s? ->" % (i.from_region.name)
    completion_text="%s is a coffee from %s. END" % (i.name, i.from_region.name)
    prompts.append({"prompt":prompt_text, "completion":completion_text})

In [78]:
prompts

[{'prompt': 'Nomad is the name of a coffee.  Body is Light.  From Region Latin America.  Roast is Blonde Roast. ->',
  'completion': 'Nomad is a Light bodied coffee that is from Latin America that has been prepared with a Blonde Roast roast. END'},
 {'prompt': 'What is the name of a good coffee with Light body? ->',
  'completion': 'Nomad is a coffee with Light body. END'},
 {'prompt': 'What is the name of a coffee from Latin America? ->',
  'completion': 'Nomad is a coffee from Latin America. END'},
 {'prompt': 'Lucy is the name of a coffee.  Body is Full.  From Region Asia Pacific.  Roast is Dark Roast. ->',
  'completion': 'Lucy is a Full bodied coffee that is from Asia Pacific that has been prepared with a Dark Roast roast. END'},
 {'prompt': 'What is the name of a good coffee with Full body? ->',
  'completion': 'Lucy is a coffee with Full body. END'},
 {'prompt': 'What is the name of a coffee from Asia Pacific? ->',
  'completion': 'Lucy is a coffee from Asia Pacific. END'},
 {'p

In [79]:
import json
with open("prompts.json", 'w') as f:
    json.dump(prompts, f)

In [60]:
#openai tools fine_tunes.prepare_data -f prompts.json
#openai api fine_tunes.create -t prompts_prepared.jsonl -m curie
#https://platform.openai.com/playground

In [42]:
openai.Completion.create(
    model="ada:ft-personal-2023-03-14-01-44-19",
    prompt="What's a good light bodied coffee from latin america?")

<OpenAIObject text_completion id=cmpl-6u13yvqRwHyusHBbun9Ee8XQAhUpU at 0x7fd208065590> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nI have recently been experimenting with various bodied brews from various lat"
    }
  ],
  "created": 1678808474,
  "id": "cmpl-6u13yvqRwHyusHBbun9Ee8XQAhUpU",
  "model": "ada:ft-personal-2023-03-14-01-44-19",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 14,
    "total_tokens": 30
  }
}

In [ ]:
openai.Completion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": "What's a light bodied washed coffee."}
    ]
)